In [ ]:
N1 = 10
N2 = 18
Nx = 35

In [ ]:
A1 = 1
A2 = 2

## imports

In [ ]:
import sympy as sp

In [ ]:
import scipy

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.autolayout"] = True
plt.rcParams["savefig.facecolor"] = (0.0, 0.0, 0.0, 0)

In [ ]:
mus = np.arange(0, 21, step = 0.1)

Axs = np.arange(0, 10, step = 0.1)

## Poisson

In [ ]:
n = sp.symbols('n', positive =  True, integer = True)

In [ ]:
mu = sp.symbols('mu', positive =  True, real = True)

In [ ]:
p = mu**n/sp.factorial(n)*sp.exp(-mu)

In [ ]:
p

## Likelihood

In [ ]:
n_1, n_2, n_x, n_tot = sp.symbols('n_1 n_2 n_x n_tot', positive =  True, integer = True)

In [ ]:
A_1, A_2, A_x = sp.symbols('A_1 A_2 A_x', positive =  True, real = True)

In [ ]:
Ntot = N1 + N2 + Nx

In [ ]:
lnL = sp.log(p.subs({mu : A_1*mu, n : n_1})) +    \
      sp.log(p.subs({mu : A_2*mu, n : n_2})) +    \
      sp.log(p.subs({mu : A_x*mu, n : n_x}))

In [ ]:
lnL

In [ ]:
print(sp.latex(lnL, ln_notation=True))

In [ ]:
lnL = sp.factor(sp.simplify(lnL))

In [ ]:
lnL

In [ ]:
print(sp.latex(lnL, ln_notation=True))

## Numerically minimise and get Hessian uncertainties

substitute all values

In [ ]:
subsA = {A_1 : A1, A_2 : A2}

In [ ]:
subsN = {n_1 : N1, n_2 : N2, n_x : Nx}

In [ ]:
subs = subsA | subsN

In [ ]:
subs

In [ ]:
lnL.subs(subs).evalf()

In [ ]:
from sympy.utilities.lambdify import lambdify

In [ ]:
#from numpy import log

In [ ]:
opt_func = lambdify([mu, A_x], lnL.subs(subs).evalf())

In [ ]:
opt_func(10, 3)

In [ ]:
opt_res = scipy.optimize.minimize(lambda x : -1*opt_func(x[0], x[1]),
                                  [10,1],
                                  hess='2-point'
                                  )

In [ ]:
opt_res

In [ ]:
mu_fit = opt_res.x[0]
Ax_fit = opt_res.x[1]

In [ ]:
mu_fit, Ax_fit

In [ ]:
opt_res.hess_inv

In [ ]:
mu_fit_std = np.sqrt(opt_res.hess_inv[0,0])
Ax_fit_std = np.sqrt(opt_res.hess_inv[1,1])

In [ ]:
print(f'simple fit results\n------------------\nmu = {mu_fit:1.2f} +/- {mu_fit_std:1.2f}\nAx = {Ax_fit:1.2f} +/- {Ax_fit_std:1.2f}')

## Margenalisation

### $\mu$

We are not really interested in $\mu$. We just want the size of the umknown tile. We find first the value for $\mu$ that maximises the likelhood. We search for the value for $\mu$ where the first derivative vanishes.

In [ ]:
lnL

In [ ]:
dlnL_dmu = sp.diff(lnL, mu)

In [ ]:
dlnL_dmu

In [ ]:
print(sp.latex(dlnL_dmu))

In [ ]:
mu_marg = sp.solve(dlnL_dmu, mu)[0]

In [ ]:
mu_marg

In [ ]:
print(sp.latex(mu_marg))

In [ ]:
lnLmarg = sp.simplify(lnL.subs(mu, mu_marg))

In [ ]:
lnLmarg

In [ ]:
print(sp.latex(lnLmarg, ln_notation=True))

In [ ]:
dlnLmarg_dAx = sp.diff(lnLmarg, A_x)

In [ ]:
dlnLmarg_dAx

In [ ]:
print(sp.latex(dlnLmarg_dAx))

In [ ]:
Ax_best = sp.solve(dlnLmarg_dAx, A_x)[0]

In [ ]:
Ax_best

In [ ]:
print(sp.latex(Ax_best))

In [ ]:
dlnLmarg_dAx_d2 = sp.diff(dlnLmarg_dAx, A_x)

In [ ]:
dlnLmarg_dAx_d2

In [ ]:
dlnLmarg_dAx_d2_Ax = sp.simplify(dlnLmarg_dAx_d2.subs({A_x : Ax_best}))

In [ ]:
dlnLmarg_dAx_d2_Ax

In [ ]:
print(sp.latex(dlnLmarg_dAx_d2_Ax))

In [ ]:
Ax_best_std = sp.sqrt(-1/dlnLmarg_dAx_d2_Ax)

In [ ]:
Ax_best_std

In [ ]:
from sympy.utilities.lambdify import lambdastr

print(lambdastr(Ax_best.subs(subsA).free_symbols, Ax_best.subs(subsA)))

print(lambdastr(Ax_best_std.subs(subsA).free_symbols, Ax_best_std.subs(subsA)))


In [ ]:
Ax_best_value = Ax_best.subs(subs).evalf()

In [ ]:
Ax_best_std_value = Ax_best_std.subs(subs).evalf()

In [ ]:
print(f'Marginalisation\n---------------\nAx = {Ax_best_value:1.2f} +/- {Ax_best_std_value:1.2f}')

In [ ]:
Delta_lnLmarg = 2*(lnLmarg.subs({A_x : Ax_best}) - lnLmarg).subs(subs).evalf()

In [ ]:
Delta_lnLmarg

In [ ]:
Delta_lnLmarg_func = lambdify(A_x, Delta_lnLmarg)

In [ ]:
Delta_lnLmarg_func(1)

In [ ]:
scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf([1, 2, 3]), df = 1)

In [ ]:
scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf([1, 2, 3]), df = 2)

In [ ]:
Ax_best_dof1 = scipy.optimize.fsolve(lambda x : Delta_lnLmarg_func(x) - scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf(1), df = 1),
                                     float(Ax_best_value)*np.array([0.5, 2]))

In [ ]:
Ax_best_dof1

In [ ]:
Ax_best_dof2 = scipy.optimize.fsolve(lambda x : Delta_lnLmarg_func(x) - scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf(1), df = 2),
                                     float(Ax_best_value)*np.array([0.5, 2]))

In [ ]:
Ax_best_dof2

In [ ]:
plt.plot(Axs,
        Delta_lnLmarg_func(Axs)
        )

plt.vlines(Ax_best_value + np.array([-Ax_best_std_value, 0, Ax_best_std_value]),
           0, 100,
           ls = 'dotted', color = 'red',
           label = 'Hessian uncertainty'
          )

plt.vlines(Ax_best_dof1,
           0, 100,
           ls = 'dotted', color = 'lightgreen',
           label = 'likelihood profile, dof = 1'
          )

plt.vlines(Ax_best_dof2,
           0, 100,
           ls = 'dotted', color = 'darkgreen',
           label = 'likelihood profile, dof = 2'
          )

plt.xlim(2,7)
plt.ylim(0,3)

plt.legend()

plt.xlabel('$A_x$')
plt.ylabel('$2(\\ln \\mathcal{L}_\\text{best} - \\ln \\mathcal{L})$')

#plt.savefig('Example_Ax_dof1.svg')
plt.savefig('Example_Ax_dof2.svg')

Note the asymmetric shape of the profile and the disagreement with a $\Delta TS = 1$. But this is the profile at the best-fit $\mu$, not the optimal $\mu$ at each point, so the profile risses quickly. That's why we need $\Delta TS = 2.3$ (dof = 2). We will see later more clearly.

### $A_x$

We can do the same with $A_x$ to get the uncertainty on $\mu$.

In [ ]:
dlnL_dA = sp.diff(lnL, A_x)

In [ ]:
dlnL_dA

In [ ]:
print(sp.latex(dlnL_dA))

In [ ]:
Ax_marg = sp.solve(dlnL_dA, A_x)[0]

In [ ]:
Ax_marg

In [ ]:
print(sp.latex(Ax_marg))

In [ ]:
lnLmarg2 = sp.simplify(lnL.subs(A_x, Ax_marg))

In [ ]:
lnLmarg2

In [ ]:
print(sp.latex(lnLmarg2, ln_notation=True))

In [ ]:
dlnLmarg2_dmu = sp.diff(lnLmarg2, mu)

In [ ]:
dlnLmarg2_dmu

In [ ]:
print(sp.latex(dlnLmarg2_dmu))

In [ ]:
mu_best = sp.solve(dlnLmarg2_dmu, mu)[0]

In [ ]:
mu_best

In [ ]:
print(sp.latex(mu_best))

In [ ]:
dlnLmarg2_dmu_d2 = sp.diff(dlnLmarg2_dmu, mu)

In [ ]:
dlnLmarg2_dmu_d2

In [ ]:
dlnLmarg2_dmu_d2_mu = sp.simplify(dlnLmarg2_dmu_d2.subs({mu : mu_best}))

In [ ]:
dlnLmarg2_dmu_d2_mu

In [ ]:
print(sp.latex(dlnLmarg2_dmu_d2_mu))

In [ ]:
mu_best_std = sp.sqrt(-1/dlnLmarg2_dmu_d2_mu)

In [ ]:
mu_best_std

In [ ]:
from sympy.utilities.lambdify import lambdastr

print(lambdastr(mu_best.subs(subsA).free_symbols, mu_best.subs(subsA)))

print(lambdastr(mu_best_std.subs(subsA).free_symbols, mu_best_std.subs(subsA)))


In [ ]:
mu_best_value = mu_best.subs(subs).evalf()

In [ ]:
mu_best_std_value = mu_best_std.subs(subs).evalf()

In [ ]:
print(f'Marginalisation\n---------------\nmu = {mu_best_value:1.2f} +/- {mu_best_std_value.subs(subs).evalf():1.2f}')

In [ ]:
Delta_lnLmarg2 = 2*(lnLmarg2.subs({mu : mu_best}) - lnLmarg2).subs(subs).evalf()

In [ ]:
Delta_lnLmarg2

In [ ]:
Delta_lnLmarg2_func = lambdify(mu, Delta_lnLmarg2)

In [ ]:
mu_best_dof1 = scipy.optimize.fsolve(lambda x : Delta_lnLmarg2_func(x) - scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf(1), df = 1),
                                     float(mu_best_value)*np.array([0.5, 2]))

In [ ]:
mu_best_dof1

In [ ]:
mu_best_dof2 = scipy.optimize.fsolve(lambda x : Delta_lnLmarg2_func(x) - scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf(1), df = 2),
                                     float(mu_best_value)*np.array([0.5, 2]))

In [ ]:
mu_best_dof2

In [ ]:
plt.plot(mus,
        Delta_lnLmarg2_func(mus)
        )

plt.vlines(mu_best_value + np.array([-mu_best_std_value, 0, mu_best_std_value]),
           0, 100,
           ls = 'dotted', color = 'red',
           label = 'Hessian uncertainty'
          )

plt.vlines(mu_best_dof1,
           0, 100,
           ls = 'dotted', color = 'lightgreen',
           label = 'likelihood profile, dof = 1'
          )

plt.vlines(mu_best_dof2,
           0, 100,
           ls = 'dotted', color = 'darkgreen',
           label = 'likelihood profile, dof = 2'
          )

plt.ylim(0,3)

plt.xlim(5, 14)

plt.legend()

plt.xlabel('$\\mu$')
plt.ylabel('$2(\\ln \\mathcal{L}_\\text{best} - \\ln \\mathcal{L})$')

plt.savefig('Example_mu_dof2.svg')

## Two-Dimensional Likelihood Profile

In [ ]:
mu_v, Ax_v = np.meshgrid(mus, Axs)

In [ ]:
Delta_lnL = sp.simplify(2*(lnL.subs({A_x : Ax_best, mu : mu_best}) - lnL))

In [ ]:
Delta_lnL

In [ ]:
# temp = (Delta_lnL.subs(subsA))

# temp

# temp.free_symbols

# from sympy.utilities.lambdify import lambdastr

# lambdastr(temp.free_symbols, temp)

In [ ]:
Delta_lnL_func = lambdify([mu, A_x], 
                          Delta_lnL.subs(subs).evalf())

In [ ]:
levels_2dof = scipy.stats.chi2.ppf(1 - 2*scipy.stats.norm.sf([0,1,2,3]), df = 2)

In [ ]:
levels_2dof

In [ ]:
plt.contourf(mu_v, Ax_v,
             Delta_lnL_func(mu_v, Ax_v),
             levels = levels_2dof 
            )

plt.hlines(Ax_best_dof1,
           np.min(mus), np.max(mus),
           ls = 'dotted', color = 'red',
           #label = 'dof = 1'
          )
plt.vlines(mu_best_dof1,
           np.min(Axs), np.max(Axs),
           ls = 'dotted', color = 'red',
           label = 'dof = 1'
          )

plt.hlines(Ax_best_dof2,
           np.min(mus), np.max(mus),
           ls = 'dotted', color = 'orange',
           #label = 'dof = 2'
          )
plt.vlines(mu_best_dof2,
           np.min(Axs), np.max(Axs),
           ls = 'dotted', color = 'orange',
           label = 'dof = 2'
          )

plt.legend()

plt.plot(10, 3, 'g+')

plt.xlabel('$\\mu$')
plt.ylabel('$A_x$')

cbar = plt.colorbar()
cbar.set_label('$2(\\ln \\mathcal{L}_\\text{best} - \\ln \\mathcal{L})$')

plt.savefig('2d_likelihood.svg')

## Improving the measurement with additional observation

Let's measure the the unknown tile again, for an expsoure three times as long.

In [ ]:
Nnew = 89
Tnew = 3

In [ ]:
n_new = sp.symbols('n_new', positive =  True, integer = True)
t_new = sp.symbols('t_new', positive =  True, real = True)

In [ ]:
subs_new = {n_new : Nnew, t_new : Tnew}

In [ ]:
p

In [ ]:
Lnew = p.subs({n : n_new, mu : t_new*mu*A_x})

In [ ]:
Lnew

In [ ]:
lnLnew = (sp.log(Lnew))

In [ ]:
lnLnew

In [ ]:
C = sp.symbols('C', positive = True, real = True)

In [ ]:
lnLnew_C = sp.factor(lnLnew.subs(A_x*mu, C))

In [ ]:
lnLnew_C

In [ ]:
C_best = sp.solve(sp.diff(lnLnew_C, C), C)[0]

In [ ]:
C_best

In [ ]:
print(sp.latex(C_best))

In [ ]:
lnLnew_min = float(lnLnew_C.subs({C : C_best} | subs_new).evalf())

In [ ]:
lnLnew = sp.factor(sp.log(Lnew))

In [ ]:
lnLnew

In [ ]:
print(sp.latex(lnLnew, ln_notation=True))

In [ ]:
lnLnew_func = lambdify([mu, A_x], lnLnew.subs(subs | subs_new).evalf())

In [ ]:
plt.contourf(mu_v, Ax_v,
             2*(lnLnew_min - lnLnew_func(mu_v, Ax_v)),
             levels = levels_2dof
            )

cbar = plt.colorbar()

plt.contour(mu_v, Ax_v,
             Delta_lnL_func(mu_v, Ax_v),
             levels = levels_2dof 
            )

plt.xlabel('$\\mu$')
plt.ylabel('$A_x$')

cbar.set_label('$2(\\ln \\mathcal{L}_\\text{best} - \\ln \\mathcal{L})$')

plt.savefig('new_likelihood.svg')

In [ ]:
lnL1 = lnL.subs(subs).evalf()

In [ ]:
lnL1

In [ ]:
print(sp.latex(lnL1, ln_notation=True))

In [ ]:
lnL2 = lnLnew.subs(subs_new).evalf()

In [ ]:
lnL2

In [ ]:
print(sp.latex(lnL2, ln_notation=True))

In [ ]:
lnLtot = lnL1 + lnL2

In [ ]:
lnLtot

In [ ]:
print(sp.latex(lnLtot, ln_notation=True))

In [ ]:
d_lnLtot_d_Ax = sp.diff(lnLtot, A_x)

In [ ]:
d_lnLtot_d_Ax

In [ ]:
d_lnLtot_d_mu = sp.diff(lnLtot, mu)

In [ ]:
d_lnLtot_d_mu

In [ ]:
print(sp.latex(d_lnLtot_d_Ax))

In [ ]:
print(sp.latex(d_lnLtot_d_mu))

In [ ]:
minimum = sp.solve([d_lnLtot_d_mu, d_lnLtot_d_Ax], mu, A_x, dict = True)

In [ ]:
minimum

In [ ]:
lnLtot_min = float(lnLtot.subs(minimum[0]))

In [ ]:
lnLtot_min

In [ ]:
lnLtot_func = lambdify([mu, A_x], lnLtot.subs(subs | subs_new).evalf())

In [ ]:
plt.contourf(mu_v, Ax_v,
             2*(lnLtot_min - lnLtot_func(mu_v, Ax_v)),
             levels = levels_2dof
            )

cbar = plt.colorbar()

plt.contour(mu_v, Ax_v,
             Delta_lnL_func(mu_v, Ax_v),
             levels = levels_2dof 
            )

plt.plot(10, 3, 'g+')

plt.xlabel('$\\mu$')
plt.ylabel('$A_x$')

cbar.set_label('$2(\\ln \\mathcal{L}_\\text{best} - \\ln \\mathcal{L})$')

plt.savefig('tot_likelihood.svg')